<a href="https://colab.research.google.com/github/carlogalli/colab-gpu/blob/main/copy_of_Julia_1_7_2_template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installation

In [ ]:
# Installation cell
%%capture
%%shell
if ! command -v julia 3>&1 > /dev/null
then
    wget -q 'https://julialang-s3.julialang.org/bin/linux/x64/1.7/julia-1.7.2-linux-x86_64.tar.gz' \
        -O /tmp/julia.tar.gz
    tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
    rm /tmp/julia.tar.gz
fi
julia -e 'using Pkg; pkg"add IJulia; precompile;"'
echo 'Done'

After you run the first cell (the the cell directly above this text), go to Colab's menu bar and select **Edit** and select **Notebook settings** from the drop down. Select *Julia 1.7* in Runtime type. You can also select your prefered harwdware acceleration (defaults to GPU). 

<br/>You should see something like this:

> ![Colab Img](https://raw.githubusercontent.com/Dsantra92/Julia-on-Colab/master/misc/julia_menu.png)

<br/>Click on SAVE
<br/>**We are ready to get going**





In [ ]:
# print Julia version
VERSION

v"1.7.2"

**The next three cells are for GPU benchmarking. If you are using this notebook for the first time and have GPU enabled, you can give it a try.** 

### Optional GPU Experiments

In [ ]:
using Pkg
Pkg.add(["BenchmarkTools", "CUDA"])
using BenchmarkTools, CUDA

if has_cuda_gpu()
  print("The GPU device is:", CUDA.device())
end

The GPU device is:CuDevice(0)

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
   Installed IrrationalConstants ─ v0.1.1
   Installed GPUArraysCore ─────── v0.1.2
   Installed Adapt ─────────────── v3.4.0
   Installed AbstractFFTs ──────── v1.2.1
   Installed BenchmarkTools ────── v1.3.1
   Installed GPUCompiler ───────── v0.16.4
   Installed TimerOutputs ──────── v0.5.21
   Installed SpecialFunctions ──── v2.1.7
   Installed CEnum ─────────────── v0.4.2
   Installed LLVMExtra_jll ─────── v0.0.16+0
   Installed BFloat16s ─────────── v0.2.0
   Installed Random123 ─────────── v1.6.0
   Installed Reexport ──────────── v1.2.2
   Installed ChainRulesCore ────── v1.15.6
   Installed GPUArrays ─────────── v8.5.0
   Installed RandomNumbers ─────── v1.5.3
   Installed LogExpFunctions ───── v0.3.18
   Installed OpenSpecFun_jll ───── v0.5.5+0
   Installed ExprTools ─────────── v0.1.8
   Installed Requires ──────────── v1.3.0
   Installed Compat ────────────── v4.3.0
   Installed Inv

In [ ]:
mcpu = rand(2^10, 2^10)
@benchmark mcpu*mcpu

BenchmarkTools.Trial: 86 samples with 1 evaluation.
 Range (min … max):  53.331 ms … 72.475 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     56.849 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   58.222 ms ±  3.796 ms  ┊ GC (mean ± σ):  0.55% ± 1.35%

      ▁    ▅█▇                                                 
  ▅▁▁▃█▆▆█▆███▁██▆█▃▁▅▆▆▁▃▃▃▃▃▃▃▃▁▁▁▁▁▁▁▃▁▁▁▁▁▃▁▁▁▁▁▁▁▃▁▁▃▁▁▃ ▁
  53.3 ms         Histogram: frequency by time        71.9 ms <

 Memory estimate: 8.00 MiB, allocs estimate: 2.

In [ ]:
println("The CuArrray operation should be much faster (~100 times) than the CPU implemenation.")
mgpu = cu(mcpu)
@benchmark CUDA.@sync mgpu*mgpu

The CuArrray operation should be much faster (~100 times) than the CPU implemenation.


BenchmarkTools.Trial: 7667 samples with 1 evaluation.
 Range (min … max):  410.814 μs …  16.987 ms  ┊ GC (min … max): 0.00% … 45.53%
 Time  (median):     630.959 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   641.415 μs ± 287.887 μs  ┊ GC (mean ± σ):  0.30% ±  0.72%

                                 ▂▆██▇▃▁                         
  ▂▁▂▁▂▁▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▃▃▄▆███████▇▆▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▁▂▂▂▂ ▃
  411 μs           Histogram: frequency by time          799 μs <

 Memory estimate: 640 bytes, allocs estimate: 32.

# Carlo's simple test
I create a simple kernel function and run it on the GPU to check the indexing is actually happening.

In [ ]:
function test(rows, cols)
  r = threadIdx().x
  c = threadIdx().y
  rows[r,c] = r
  cols[r,c] = c
  return
end



test (generic function with 2 methods)

In [23]:
n=5
a = CUDA.zeros(n,n)
b = CUDA.zeros(n,n)
@cuda threads=(3,2) test(a,b)
[a; b]

10×5 CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}:
 1.0  1.0  0.0  0.0  0.0
 2.0  2.0  0.0  0.0  0.0
 3.0  3.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 1.0  2.0  0.0  0.0  0.0
 1.0  2.0  0.0  0.0  0.0
 1.0  2.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0